# Homework 2

In [1]:
import numpy as np
import pandas as pd
import html5lib

import requests
from bs4 import BeautifulSoup

## Paramethers for requests:
    1- ww_x_UNITE_ACAD - Section (Informatique?)  =  249847
    2- ww_x_PERIODE_ACAD - Semester wanted        =  978181
    3- ww_i_reportModel - General parameter =  133685247

In [2]:
unite_academique=249847
periode_academique=[978181, 978187,978195,39486325,123455150,213637754,213637922,213638028,355925344]
report_model=133685247



In [3]:
parameters={'ww_i_reportModel':'133685247','ww_i_reportModelXsl':'133685270','ww_x_UNITE_ACAD':'249847'}

In [4]:
base_all_gps='http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270&zz_x_UNITE_ACAD=Informatique'

In [5]:
informatique = {'ww_x_UNITE_ACAD': '249847'}
r = requests.get(base_all_gps,params=informatique)

In [6]:
r.text

'<html><head><META http-equiv="Content-Type" content="text/html; charset=iso-8859-1"><div></div><title></title><script type="text/javascript" src="GEDPUBLICREPORTS.txt?ww_x_path=Gestac.Base.Palette_js&amp;ww_c_langue=fr"></script><link rel="stylesheet" type="text/css" href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.Style"><link rel="stylesheet" type="text/css" href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.StyleNavigator"></head><body bgcolor="#ffffff" marginheight="0" marginwidth="5" link="#666666" vlink="#666666" alink="#666666"><div class="filtres"><form method="GET" name="f" action="!GEDPUBLICREPORTS.filter"><input type="hidden" name="ww_b_list" value="1"><input type="hidden" name="ww_i_reportmodel" value="133685247"><input type="hidden" name="ww_c_langue" value=""><h1 id="titre">Liste des étudiants inscrits par semestre</h1><table border="0" id="format"><tr><th>Format:</th></tr><tr><td><input type="radio" name="ww_i_reportModelXsl" value="133685270" checked>html</td></t

In [7]:
semester_bachelor=['Bachelor semestre 1','Bachelor semestre 2','Bachelor semestre 3','Bachelor semestre 4','Bachelor semestre 5','Bachelor semestre 6']
semester_master=['Master semestre 1','Master semestre 2','Master semestre 3']
years=['2007-2008','2008-2009','2009-2010','2010-2011','2011-2012','2012-2013','2013-2014','2014-2015','2015-2016','2016-2017']

In [8]:
# Generate dictionary
gps_dictionary={}
for year in years:
    for semester in semester_bachelor:
        gps_dictionary['Informatique, '+year+', '+semester]=0
    for semester in semester_master:
        gps_dictionary['Informatique, '+year+', '+semester]=0

In [9]:
# Generate dictionary
main_dictionary={}
for year in years:
    main_dictionary[year]=dict()
    for item in semester_bachelor:
        main_dictionary[year][item]=0
    for item in semester_master:
        main_dictionary[year][item]=0
    

In [10]:
soup=BeautifulSoup(r.text, "lxml")

In [11]:
# Parse the parameter "periode_academique"

parse_years=soup.find_all('option')

for year in main_dictionary:
    for item in parse_years:
        if year==item.string:
            main_dictionary[year]['periode_academique']=str(item).split("\"")[1]

In [12]:
tag=soup.body

gps_list=tag.find_all('a')

In [13]:
# Process the GPS list
# Remove the ones without "Informatique", "Master" or "Bachelor"
# Hard preprocessing to fast eliminate the unwanted strings
gps_processed=[]
key1='Informatique'
key2='Bachelor'
key3='Master'
for i in gps_list:
    if key1 in i.contents[0] and (key2 in i.contents[0] or key3 in i.contents[0]):
        gps_processed.append(i)


In [14]:
# Keep only the right ones - get the GPS in the main_dictionary
for year in main_dictionary:
    for semester in main_dictionary[year]:
        for item in gps_processed:
            if 'Informatique, '+year+', '+semester==item.contents[0]:
                gps=str(item).split("'")[1].split("=")[1]
                main_dictionary[year][semester]=gps
print(main_dictionary)

{'2015-2016': {'Master semestre 2': '1897033287', 'Bachelor semestre 5': '1650771974', 'Bachelor semestre 3': '1744377949', 'Master semestre 3': '1745490470', 'Bachelor semestre 1': '1897032870', 'Bachelor semestre 4': '1744377976', 'Bachelor semestre 2': '1897032920', 'periode_academique': '213638028', 'Master semestre 1': '1897033225', 'Bachelor semestre 6': '1650772010'}, '2014-2015': {'Master semestre 2': '1745490396', 'Bachelor semestre 5': '1378362202', 'Bachelor semestre 3': '1650771920', 'Master semestre 3': '1650773371', 'Bachelor semestre 1': '1744377893', 'Bachelor semestre 4': '1650771947', 'Bachelor semestre 2': '1744377921', 'periode_academique': '213637922', 'Master semestre 1': '1745490358', 'Bachelor semestre 6': '1378362238'}, '2007-2008': {'Master semestre 2': '71450725', 'Bachelor semestre 5': '19561980', 'Bachelor semestre 3': '39494788', 'Master semestre 3': '39495424', 'Bachelor semestre 1': '71297531', 'Bachelor semestre 4': '39494823', 'Bachelor semestre 2': '7

# Get the tables

In [15]:
base_request='http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html'

In [16]:
parameters={'ww_i_reportModel':'133685247','ww_i_reportModelXsl':'133685270'
            ,'ww_x_UNITE_ACAD':'249847','ww_x_PERIODE_ACAD':0,'ww_x_GPS':0}

In [88]:
bachelor_data={x:dict() for x in main_dictionary}
master_data={x:dict() for x in main_dictionary}
for year in main_dictionary:
    parameters['ww_x_PERIODE_ACAD']=main_dictionary[year]['periode_academique']
    for semester in main_dictionary[year]:
        if semester=='periode_academique':
            continue
        parameters['ww_x_GPS']=main_dictionary[year][semester]
        
        # Post the request
        req = requests.get(base_request,params=parameters)
        
        # Get the html file with BeautifulSoap (maybe not needed)
        #html_table=BeautifulSoup(req.text, "lxml")
        
        # Import and process the dataframe with pandas
        df=pd.read_html(str(req.text),skiprows=1,header=0)[0]
        df.columns=['Title','Name','OrienB','OrienM','Spec','Unuseful1','Mineur','State','Exchange','ExchangeSchool','Sciper','Unuseful2']
        del df['Title'],df['OrienB'],df['OrienM'],df['Spec'],df['Unuseful1'],df['State'],df['Exchange'],df['ExchangeSchool'],df['Unuseful2']
        df['Year']=year
        
        type_semester=semester.split(' ')[0]
        semester_number=int(semester.split(' ')[2])
        df['Semester']=semester_number
        df=df.set_index(['Sciper','Year','Semester'])

        if type_semester=="Bachelor":
            bachelor_data[year][semester_number]=df
        elif type_semester=="Master":
            master_data[year][semester_number]=df
        else:
            print(type_semester)
            raise("Error parsing file: type semester not acceptable")
    print("Year "+year+" parsed")

Year 2015-2016 parsed
Year 2014-2015 parsed
Year 2007-2008 parsed
Year 2016-2017 parsed
Year 2009-2010 parsed
Year 2013-2014 parsed
Year 2011-2012 parsed
Year 2008-2009 parsed
Year 2010-2011 parsed
Year 2012-2013 parsed


In [89]:
# Check of correct parsing of all the semesters
bachelor_data_semesters=0
master_data_semesters=0
for year in master_data:
    master_data_semesters+=len(master_data[year])
for year in bachelor_data:
    bachelor_data_semesters+=len(bachelor_data[year])
if (bachelor_data_semesters+master_data_semesters!=90):
    raise("Some semester has not been correctly parsed")

In [128]:
bachelor_tables=[]
for year in bachelor_data:
    for semester in bachelor_data[year]:
        bachelor_tables.append(bachelor_data[year][semester])
total=pd.concat(bachelor_tables,axis=0,join='outer')
total=total.reset_index()
t=total.groupby('Sciper')

(147008,       Sciper       Year  Semester         Name  Mineur
2403  147008  2009-2010         3  Good Xavier     NaN
2457  147008  2009-2010         4  Good Xavier     NaN
4259  147008  2008-2009         1  Good Xavier     NaN
4348  147008  2008-2009         2  Good Xavier     NaN
5011  147008  2010-2011         5  Good Xavier     NaN
5069  147008  2010-2011         6  Good Xavier     NaN)
(153762,       Sciper       Year  Semester          Name  Mineur
2094  153762  2007-2008         5  Conus Johann     NaN
2139  153762  2007-2008         6  Conus Johann     NaN)
(154157,       Sciper       Year  Semester                              Name  Mineur
2083  154157  2007-2008         5  Andriambololona Riana Miarantsoa     NaN)
(159516,       Sciper       Year  Semester         Name  Mineur
2117  159516  2007-2008         5  Raja Yanick     NaN
2159  159516  2007-2008         6  Raja Yanick     NaN)
(159998,       Sciper       Year  Semester         Name  Mineur
2106  159998  2007-2008   

In [136]:
s=0
for i in t:
    s=i
print(type(s))

<class 'tuple'>
